In [2]:
import pandas as pd
import numpy as np
import uuid
import pickle
import os
import configparser
import pymysql
import multiprocessing as mp
import sys
from imp import reload
from tqdm import tqdm
# from pyspark.ml.fpm import FPGrowth
from Code import data_generator_new
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [3]:
reload(data_generator_new)

<module 'Code.data_generator_new' from '/home/zijun.wu/Tag_graph/Code/data_generator_new.py'>

### 数据探查

In [ ]:
data_raw = pd.read_sql("select comp_id, label_name, classify_id, label_type, label_type_num from company_tag_info_latest0703", con=data_generator_new.db)

In [ ]:
len(data_raw)

In [ ]:
comp_id_dict = data_raw.comp_id.drop_duplicates().reset_index(drop=True).reset_index() \
    .rename(index=str, columns={"index": "comp_int_id"})
data_raw = data_raw.merge(comp_id_dict, how="left", left_on="comp_id", right_on="comp_id")

In [ ]:
data_raw.head(3)

In [ ]:
ctag_table = data_raw[data_raw.classify_id != 4]

In [ ]:
ctag_agg = nctag_table.groupby("label_name").agg({"comp_id": "count"})

In [ ]:
ctag_agg.sort_values(by="comp_id", ascending=False)

In [ ]:
nctag_table = data_raw[data_raw.classify_id == 4]

In [ ]:
nctag_agg = nctag_table.groupby("label_name").agg({"comp_id": "count"}).reset_index()

In [ ]:
nctag_agg["word_len"] = nctag_agg.label_name.apply(lambda x: len(str(x)))

In [ ]:
nctag_agg.sort_values(by="comp_id", ascending=False).head(100)

In [ ]:
nctag_agg[nctag_agg.comp_id >= 150].groupby("word_len").agg({"label_name": "count"})

In [ ]:
nctag_agg[(nctag_agg.comp_id >= 150) & (nctag_agg.word_len > 3)]

In [ ]:
nctag_to_keep = nctag_agg[(nctag_agg.comp_id >= 150) & (nctag_agg.comp_id <= 1000000) & (nctag_agg.word_len >= 4)]

In [ ]:
nctag_to_keep

In [ ]:
nctag_table = nctag_table[nctag_table.label_name.isin(nctag_to_keep.label_name)]

In [ ]:
nctag_table.head(2)

In [ ]:
ctag_table.head(2)

### 检测comp_tag函数

In [ ]:
data_generator_new.comp_tag()

### 检测data_aggregator函数，返回概念、非概念各自聚合公司的结果，以及公司标签关联、公司总数

In [ ]:
data_generator_new.data_aggregator()

### 计算公司标签全集

In [ ]:
data_generator_new.properties()

### 计算标签关联

In [ ]:
data_generator_new.tag_tag4(batch_size=4, process_num=8)

### 处理成导入mysql和neo4j的文件

In [4]:
data_generator_new.neo4j_merge()

/home/zijun.wu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2909: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Points saved!
Relations saved!


0

### 导入图数据库

In [5]:
data_generator_new.to_graph_database()

Data imported to neo4j!


0

In [7]:
spark

In [8]:
sc

<SparkContext master=local[*] appName=PySparkShell>

### 全流程测试

In [ ]:
!python Code/pipline_new.py

In [ ]:
1

### 智能网联汽车关联的top1000概念非概念标签

In [ ]:
relations = pd.read_csv("../Data/Output/Tag_graph/all_relations.csv", header=None)

In [ ]:
relations.columns = ["src_id", "target_id", "rel_value", "rel_type"]

In [ ]:
relations.head(1)

In [ ]:
tag_dict = pickle.load(open("../Data/Output/Tag_graph/tag_dict.pkl", "rb"))
tag_dict_reverse = dict(zip(tag_dict.values(), tag_dict.keys()))

In [ ]:
target_tag = tag_dict.get("智能网联汽车")

In [ ]:
target_tag

In [ ]:
linked_ctag_top_1000 = relations[(relations.src_id == target_tag) & (relations.rel_type == "ctag_ctag")].sort_values(by="rel_value", ascending=False)

In [ ]:
len(linked_ctag_top_1000)

In [ ]:
_linked_ctag_top_1000 = relations[(relations.src_id == target_tag) & (relations.rel_type == "ctag_ctag")].sort_values(by="rel_value", ascending=False)

In [ ]:
len(_linked_ctag_top_1000)

In [ ]:
linked_nctag_top_1000 = relations[(relations.target_id == target_tag) & (relations.rel_type == "ctag_ctag")].sort_values(by="rel_value", ascending=False)[:1000]

In [ ]:
len(linked_nctag_top_1000)

In [ ]:
_linked_nctag_top_1000 = relations[(relations.src_id == target_tag) & (relations.rel_type == "ctag_ntag")].sort_values(by="rel_value", ascending=False)[:1000]

In [ ]:
linked_ctag_top_1000["target_tag_name"] = linked_ctag_top_1000.targe_id.apply(lambda x: tag_dict_reverse.get(x))[:1000]

In [ ]:
linked_nctag_top_1000["target_tag_name"] = linked_nctag_top_1000.targe_id.apply(lambda x: tag_dict_reverse.get(x))[:1000]

# MinHash test

In [2]:
from pyspark.ml.feature import CountVectorizer, MinHashLSH
from pyspark.sql.functions import col

In [3]:
%%time
tag_comps_aggregated = pickle.load(open("../Data/Output/Tag_graph/tag_comps_aggregated.pkl", "rb"))

CPU times: user 4.6 s, sys: 1.4 s, total: 6 s
Wall time: 5.99 s


In [4]:
tag_comps_aggregated.head(3)

,comp_int_id,type,tag_uuid
0,"{1671170, 5406730, 1179663, 3735567, 262169, 7...",1,a2f6e88e9a65510ba21ce44965ca7b36
1,"{1671170, 3637262, 589847, 2129945, 851998, 16...",1,b5426c809798598cb5735dcb6a7c17c8
2,"{2275332, 5376008, 1241096, 3989514, 5883914, ...",1,299f1692f39a54989bb7a754533d7e3e


In [5]:
len(tag_comps_aggregated)

17455

In [6]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [7]:
tag_comps_aggregated["comp_int_id_list"] = tag_comps_aggregated.comp_int_id.apply(lambda x: [str(w) for w in x])

In [8]:
tag_comps_aggregated_spark = spark.createDataFrame(tag_comps_aggregated[["tag_uuid", "comp_int_id_list"]])

In [9]:
tag_comps_aggregated_spark.show(3)

+--------------------+--------------------+
|            tag_uuid|    comp_int_id_list|
+--------------------+--------------------+
|a2f6e88e9a65510ba...|[1671170, 5406730...|
|b5426c809798598cb...|[1671170, 3637262...|
|299f1692f39a54989...|[2275332, 5376008...|
+--------------------+--------------------+
only showing top 3 rows



In [10]:
tag_comps_aggregated_spark.persist()

DataFrame[tag_uuid: string, comp_int_id_list: array<string>]

In [11]:
%%time
print("Sparse Vectorizing")
cv = CountVectorizer(inputCol="comp_int_id_list", outputCol="sparse_vector", minDF=1, minTF=1, vocabSize=7000000)
cv_model = cv.fit(tag_comps_aggregated_spark)

Sparse Vectorizing
CPU times: user 24.1 ms, sys: 5.62 ms, total: 29.7 ms
Wall time: 52.4 s


In [12]:
%%time
tag_comps_sparse = cv_model.transform(tag_comps_aggregated_spark)
tag_comps_sparse.show(3)
mh = MinHashLSH(inputCol="sparse_vector", outputCol="hashes", seed=12345, numHashTables=128)
print("Fitting lsh model")
mh_model = mh.fit(tag_comps_sparse)
tag_comps_sparse.persist()

+--------------------+--------------------+--------------------+
|            tag_uuid|    comp_int_id_list|       sparse_vector|
+--------------------+--------------------+--------------------+
|a2f6e88e9a65510ba...|[1671170, 5406730...|(6663224,[106,195...|
|b5426c809798598cb...|[1671170, 3637262...|(6663224,[26,106,...|
|299f1692f39a54989...|[2275332, 5376008...|(6663224,[339,690...|
+--------------------+--------------------+--------------------+
only showing top 3 rows

Fitting lsh model
CPU times: user 40 ms, sys: 13.3 ms, total: 53.3 ms
Wall time: 1min 49s


In [ ]:
%%time
print("Similarity cross join")
result = mh_model.approxSimilarityJoin(tag_comps_sparse, tag_comps_sparse, 1.0, distCol="JaccardDistance").filter("JaccardDistance != 1.0")
result.persist()
result.count()
result.show(3)
result_clean = result.select(
    col("datasetA.tag_uuid").alias("tag1"),
    col("datasetB.tag_uuid").alias("tag2"),
    col("JaccardDistance"))
result_clean.show(3)

In [ ]:
result.show(3)

In [3]:
df = pd.DataFrame([[1,2],[3,4],[5,6], [7,8], [9,0]])

In [4]:
df

,0,1
0,1,2
1,3,4
2,5,6
3,7,8
4,9,0


In [7]:
x = np.array_split(df.index, 3)

In [9]:
x

[Int64Index([0, 1], dtype='int64'),
 Int64Index([2, 3], dtype='int64'),
 Int64Index([4], dtype='int64')]

In [32]:
df.iloc[x[2].tolist()]

KeyError: '[4] not in index'

In [25]:
label_chains_raw = pickle.load(open("../Data/Output/Tag_graph/label_chains_raw.pkl", "rb"))

In [ ]:
label_chains_raw

In [9]:
%connect_info

{
  "shell_port": 60890,
  "iopub_port": 52076,
  "stdin_port": 41499,
  "control_port": 37182,
  "hb_port": 48673,
  "ip": "127.0.0.1",
  "key": "14bc2e5c-738e58d957637bd85dcbfc26",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-de6527ec-c3b8-4a5a-a40c-c96e86816567.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
